In [2]:
import pandas as pd

In [30]:
# 파일 경로 (사용자 환경에 맞게 수정)
path_floating = r'C:\hanchon\data\raw\population\서울시 상권분석서비스(길단위인구-행정동).csv'
path_resident = r'C:\hanchon\data\raw\population\서울시 상권분석서비스(상주인구-행정동).csv'
path_working = r'C:\hanchon\data\raw\population\서울시 상권분석서비스(직장인구-행정동).csv'

# 데이터 불러오기
df_floating = pd.read_csv(path_floating, encoding='cp949')
df_resident = pd.read_csv(path_resident, encoding='cp949')
df_working = pd.read_csv(path_working, encoding='cp949')

print("--- 유동인구 데이터 ---")
print(df_floating.head(1))
print("\n--- 거주인구 데이터 ---")
print(df_resident.head(1))
print("\n--- 직장인구 데이터 ---")
print(df_working.head(1))

--- 유동인구 데이터 ---
   기준_년분기_코드    행정동_코드 행정동_코드_명  총_유동인구_수  남성_유동인구_수  여성_유동인구_수  \
0      20251  11440730     성산2동   6703555    2965052    3738503   

   연령대_10_유동인구_수  연령대_20_유동인구_수  연령대_30_유동인구_수  연령대_40_유동인구_수  ...  \
0        1067104        1068243        1190237        1156579  ...   

   시간대_14_17_유동인구_수  시간대_17_21_유동인구_수  시간대_21_24_유동인구_수  월요일_유동인구_수  \
0            682163           1015386            913222      948744   

   화요일_유동인구_수  수요일_유동인구_수  목요일_유동인구_수  금요일_유동인구_수  토요일_유동인구_수  일요일_유동인구_수  
0      940056      943142      941600      942451      983732     1003830  

[1 rows x 25 columns]

--- 거주인구 데이터 ---
   기준_년분기_코드    행정동_코드 행정동_코드_명  총_상주인구_수  남성_상주인구_수  여성_상주인구_수  \
0      20241  11740590     암사3동     17136       8416       8720   

   연령대_10_상주인구_수  연령대_20_상주인구_수  연령대_30_상주인구_수  연령대_40_상주인구_수  ...  \
0           3437           1763           2030           3124  ...   

   남성연령대_60_이상_상주인구_수  여성연령대_10_상주인구_수  여성연령대_20_상주인구_수  여성연령대_30_상주인구_수  \
0                19

In [31]:
# 모든 데이터프레임에 대해 연도/분기 분리 작업을 한 번에 수행
for df in [df_floating, df_resident, df_working]:
    # 1. '기준_년분기_코드'를 문자열로 변환
    df['년분기_str'] = df['기준_년분기_코드'].astype(str)
    
    # 2. 앞 4자리는 'year', 뒤 1자리는 'quarter'로 분리
    df['year'] = df['년분기_str'].str[:4].astype(int)
    df['quarter'] = df['년분기_str'].str[4:].astype(int)
    
    # 3. 임시로 사용한 '년분기_str' 컬럼 삭제
    df.drop(columns=['년분기_str'], inplace=True)

print("--- 연도/분기 분리 후 유동인구 데이터 ---")
print(df_resident[['year', 'quarter']].head())

--- 연도/분기 분리 후 유동인구 데이터 ---
   year  quarter
0  2024        1
1  2024        1
2  2024        1
3  2024        1
4  2024        1


In [32]:
# 실제 데이터의 행정동
target_dong = '삼성동' 

# '행정동_코드_명' 컬럼이 '삼성동'인 데이터만 정확히 추출합니다.
samsung_floating = df_floating[df_floating['행정동_코드_명'] == target_dong].copy()
samsung_resident = df_resident[df_resident['행정동_코드_명'] == target_dong].copy()
samsung_working = df_working[df_working['행정동_코드_명'] == target_dong].copy()

print(f"--- '{target_dong}' 필터링 후 shape ---")
print("유동인구:", samsung_floating.shape)
print("거주인구:", samsung_resident.shape)
print("직장인구:", samsung_working.shape)

--- '삼성동' 필터링 후 shape ---
유동인구: (26, 27)
거주인구: (26, 29)
직장인구: (26, 26)


In [33]:
# 'samsung_floating', 'samsung_resident', 'samsung_working'에 대해 각각 수행합니다.

# 1. 2023년 이후 데이터만 선택
samsung_floating_filtered = samsung_floating[samsung_floating['year'] >= 2023].copy()
samsung_resident_filtered = samsung_resident[samsung_resident['year'] >= 2023].copy()
samsung_working_filtered = samsung_working[samsung_working['year'] >= 2023].copy()

# 2. 2025년 3분기를 초과하는 데이터는 제외
#    (2025년이면서 4분기인 데이터를 제외하는 조건)
samsung_floating_filtered = samsung_floating_filtered[~((samsung_floating_filtered['year'] == 2025) & (samsung_floating_filtered['quarter'] > 3))]
samsung_resident_filtered = samsung_resident_filtered[~((samsung_resident_filtered['year'] == 2025) & (samsung_resident_filtered['quarter'] > 3))]
samsung_working_filtered = samsung_working_filtered[~((samsung_working_filtered['year'] == 2025) & (samsung_working_filtered['quarter'] > 3))]


# 3. 결과 확인 (이제 11개 행만 남아야 합니다)
print("--- 기간 필터링 후 shape ---")
print("유동인구:", samsung_floating_filtered.shape)
print("거주인구:", samsung_resident_filtered.shape)
print("직장인구:", samsung_working_filtered.shape)

# 데이터가 잘 걸러졌는지 정렬해서 확인
print("\n--- 최종 필터링된 유동인구 데이터 ---")
print(samsung_floating_filtered.sort_values(by=['year', 'quarter']))

--- 기간 필터링 후 shape ---
유동인구: (10, 27)
거주인구: (10, 29)
직장인구: (10, 26)

--- 최종 필터링된 유동인구 데이터 ---
       기준_년분기_코드    행정동_코드 행정동_코드_명  총_유동인구_수  남성_유동인구_수  여성_유동인구_수  \
10544      20231  11620745      삼성동   7115705    3622333    3493372   
10943      20232  11620745      삼성동   7279980    3639094    3640885   
8624       20233  11620745      삼성동   7123932    3596604    3527330   
8952       20234  11620745      삼성동   7062246    3564914    3497331   
630        20241  11620745      삼성동   7125849    3642018    3483829   
993        20242  11620745      삼성동   7246114    3653456    3592659   
1517       20243  11620745      삼성동   7116023    3586935    3529086   
1716       20244  11620745      삼성동   7029660    3546129    3483531   
115        20251  11620745      삼성동   6935735    3514419    3421315   
2202       20252  11620745      삼성동   6929967    3470325    3459642   

       연령대_10_유동인구_수  연령대_20_유동인구_수  연령대_30_유동인구_수  연령대_40_유동인구_수  ...  \
10544         631758        1352676        1092054

In [ ]:
# Step 4: 분석에 필요한 컬럼 선택 및 이름 변경
# (변수명은 필터링된 데이터프레임 이름으로 수정)
# 유동인구 데이터 정리
samsung_floating_final = samsung_floating_filtered[['year', 'quarter', '총_유동인구_수']]
samsung_floating_final.rename(columns={'총_유동인구_수': 'floating_pop'}, inplace=True)

# 거주인구 데이터 정리
samsung_resident_final = samsung_resident_filtered[['year', 'quarter', '총_상주인구_수']]
samsung_resident_final.rename(columns={'총_상주인구_수': 'resident_pop'}, inplace=True)

# 직장인구 데이터 정리
samsung_working_final = samsung_working_filtered[['year', 'quarter', '총_직장_인구_수']]
samsung_working_final.rename(columns={'총_직장_인구_수': 'working_pop'}, inplace=True)

# Step 5: 3개 데이터를 하나로 병합(Merge)
# 1. 유동인구와 거주인구 데이터를 병합
df_population = pd.merge(samsung_floating_final, samsung_resident_final, on=['year', 'quarter'], how='left')

# 2. 위 결과에 직장인구 데이터를 병합
df_population = pd.merge(df_population, samsung_working_final, on=['year', 'quarter'], how='left')

print("\n--- 최종 병합된 인구 데이터 ---")
print(df_population.shape) # (10, 5)가 되어야 합니다.
print(df_population)

# Step 6: 결과 저장
output_path = r'C:\hanchon\data\processed\population_samsung_quarterly.csv'
df_population.to_csv(output_path, index=False)

print(f"\n인구 데이터 가공 완료! 경로: {output_path}")


--- 최종 병합된 인구 데이터 ---
(10, 5)
   year  quarter  floating_pop  resident_pop  working_pop
0  2025        1       6935735         19970          880
1  2024        1       7125849         19970          713
2  2024        2       7246114         19970          713
3  2024        3       7116023         19970          713
4  2024        4       7029660         19970          880
5  2025        2       6929967         19970          880
6  2023        3       7123932         25864          886
7  2023        4       7062246         19970          713
8  2023        1       7115705         25864          886
9  2023        2       7279980         25864          886

✅ 인구 데이터 가공 완료! 경로: C:\hanchon\data\processed\population_samsung_quarterly.csv


C:\Users\leesk\AppData\Local\Temp\ipykernel_67372\2631205653.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  samsung_floating_final.rename(columns={'총_유동인구_수': 'floating_pop'}, inplace=True)
C:\Users\leesk\AppData\Local\Temp\ipykernel_67372\2631205653.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  samsung_resident_final.rename(columns={'총_상주인구_수': 'resident_pop'}, inplace=True)
C:\Users\leesk\AppData\Local\Temp\ipykernel_67372\2631205653.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/